                                             Workshop 8: Sadra Sadeh

# Introduction to modelling neuronal dynamics in networks with excitation and inhibition

The aim of the today's lecture was to introduce you to modelling neuronal responses across multiple scales. We saw that the dynamics of the brain and its functional and dysfunctional activity can be modelled across different levels, from single neurons to networks of neurons. 

In this tutorial, we focus on one class of models introduced in the lecture, namely Wilson-Cowan network models. The Wilson-Cowan model describes the general dynamics of interacting excitatory and inhibitory neuronal populations. We use this model to gain insight into the evolution of neural activity during transient dynamics, as well as in the stationary state (in equilibrium):

- **Transient dynamics**: the dynamics of responses to external perturbations or intrinsic fluctuations;
- **Stationary State**: the activity of neuron and networks when settled into a rather stable state.

In the workshop today, we will start to teach you how to model and understand these aspects of neural dynamics, and how to analyse large-scale networks composed of excitatory and inhibitory units.


Before starting to implement the model, we need to import the modules that you will need in the workshop.

In [ ]:
import numpy as np 
from scipy import fftpack

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sb
sb.set_theme("talk")
sb.set_style("whitegrid")

We then start by implementing the two equations described in the lecture by the Wilson-Cowan model.

There are two equations each describing the dynamics of one population.

The first one, describing the dynamics of the excitatory (E) population:

- $\tau_E \, dr_E/dt = -r_E + g(w_{EE} \, r_E - w_{EI} \, r_I + I_E)$

And the second one, describing the dynamics of the inhibitory (I) population:

- $\tau_E \, dr_E/dt = -r_I + g(w_{IE} \, r_E - w_{II} \, r_I + I_I)$

Familiarize yourself with different terms, as we need to play with them in the following:

- $r_E$ and $r_I$: activity of excitatory (E) and inhibitory (I) populations
- $dr$: changes in the activity at each time step
- $w$: weights of recurrent connections between the two population (with 4 possibilities)
- $\tau$: time constant of integration (effectively, how fast or slow each population respond to the input)
- $g$: activation function (which describes how the input is transferred to output)

For the activation function, we choose a linear-rectified function.
This activation function rectifies (=0) any negative input ($g(x)=0$ if $x<0$), and returns the same value for any positive value (hence linear: $g(x)=x$ if $x>0$).
The following function implements such an activation function.  

In [ ]:
# - activation function (linear-rectified)
def act_fun(z): 
    return z *(z>0)

## A solver for Wilson-Cowan equations

Now we need to solve the main equations. 

These are two "differential equations", meaning that they are equations that describe changes in some parameters as a function of their current values: $dx = f(x)$. To solve such equations, we need to update them in "steps": the value of $x$ in the next step ($x(n+1)$) is obtained from its previous value plus the "differential": $x(n+1) = x(n) + dx = x(n) + f(x)$. 

Since we are interested in temporal dynamics, these steps are "time steps". We use Python to write a for loop which runs over time (in time steps, $dt$, that we define). At each time step, we calculate the differential ($dx$ - or $dr$, from Wilson-Cowan equations), and update the parameters of interest (in this case, the activity of excitation and inhibition, $r_E$ and $r_I$). In the next step, the differentials are calculated from the updated parameters, and so on and so forth. This is called the Euler method to solve a differential equation (there are more complicated methods too, for more complex equations, but this would work for us for now).  

The function below is a solver that uses this method to solve Wilson-Cowan equations.

In [ ]:
# - solver function (for Wilson-Cowan equations)
def WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end):

    T = np.arange(0, T_end, dt);

    r_E = np.zeros(len(T));
    r_I = np.zeros(len(T));

    #I_E = .5+pl.rand(len(T));
    #I_I = 1+pl.rand(len(T));

    for i in range(len(T)-1):  
        dr_E = dt/tau_E * (-r_E[i] + act_fun(w_EE * r_E[i] + w_EI *r_I[i] + I_E));
        dr_I = dt/tau_I * (-r_I[i] + act_fun(w_IE * r_E[i] + w_II *r_I[i] + I_I));

        r_E[i+1] = r_E[i] + dr_E
        r_I[i+1] = r_I[i] + dr_I

    return r_E, r_I, T

As you can see, the function receives multiple inputs.
- First, inputs (constant values) to excitation and inhibition: I_E and I_I.
- Second, the weights of connection between excitatory and inhibitory neurons: w_EE, w_EI, w_IE, w_II.
- Third, the time constants of integration for different populations: tau_E, tau_I.
- Finally, two parameters describing the time steps of integration: dt which determines the time step (or time resolution of integration); and T_end, which specifies for how long the equations are solved.   

In return, the function gives us two arrays (r_E and r_I) corresponding to the activity of excitatory and inhibitory populations, and the array of time points (T) for which the equations were solved (all with the total length of T_end/dt).

In the following, we use the function to solve the equations for these parameters:
- I_E = 1; I_I = 1
- w_EE = 0.5; w_IE = 0.5; w_EI = -0.5; w_II = -0.5
- tau_E = 10; tau_I = 10
- dt = 0.1; T_end = 1000

[Note that dt should be always chosen to be an order of magnitude smaller than characteristic time scale of dynamics, namely tau_E and tau_I here.]

In [ ]:
dt = 0.1;
T_end = 200;

tau_E = 10;
tau_I = 10;

w_EE = 0.5;
w_IE = 0.5; 
w_EI = -0.5; 
w_II = -0.5;

I_E = 1; 
I_I = 1;

[r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);

print(T)
print(r_E)
print(r_I)

We can visualize the activity of the two populations by plotting them against time.
What you see is the convergence of activity of two populations to "steady state" values .. (transient and s.s.)

In [ ]:
plt.figure()
plt.plot(T, r_E, 'r-')
plt.plot(T, r_I, 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('Time')
plt.ylabel('Activity')


## Effect of feedforward input on neuronal dynamics

You can now experiment with different values of the function / equations and observe their effects on neuronal activity. 

First start by changing the inputs.

Change the input to excitatory and inhibitory neurons, and compare the responses to the following inputs:

- (I_E, I_I) = (1, 1)
- (I_E, I_I) = (0.5, 1) 
- (I_E, I_I) = (1, 0.5)
- (I_E, I_I) = (2, 2)

How do the dynamics of the activity change in each case?
Describe this during transient responses, and when the activity reaches its "steady state" (given the time constants of integration, the current simulation time is long enough such that the activity at the end of simulation can be assumed to be in the steady state).

Can you predict the steady state value of activity from the equations?

In [ ]:
# CODE HERE


We can plot the steady state responses of E and I to varying inputs. 
The following code ..
steady state response is taken as the last value

when do the two lines cross? why?

In [ ]:
I_E_range = np.arange(0, 2, .1); 
I_I = 1.0;

r_E_range = np.zeros(I_E_range.shape);
r_I_range = np.zeros(I_E_range.shape);
for i, I_E in enumerate(I_E_range):
    [r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);
    r_E_range[i] = r_E[-1];
    r_I_range[i] = r_I[-1];

plt.figure()
plt.plot(I_E_range, r_E_range, 'r-')
plt.plot(I_E_range, r_I_range, 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('I_E')
plt.ylabel('Activity (steady state)')

Change the above code to plot the steady state responses to varying input to inhibition.

Do you see the same dependence? Describe and explain your observation.

In [ ]:
# CODE HERE
I_I_range = np.arange(0, 2, .1); 
I_E = 1;


## Effect of recurrent coupling on neuronal dynamics

Now let's experiment with the recurrent couplings.

In the following code, we set the inputs to initial values (I_E = I_I = 1), and simulate the activity of excitatory and inhibitory units for different values of recurrent connectivity.

We first investigate the effect of excitatory-to-excitatory connectivity (w_EE). To do that, we turn off all other recurrent connections, namely w_EI = w_IE = w_II = 0, and change the value of w_EE.

The code below is written for w_EE = 0.5. Change the value of w_EE to smaller and larger values and compare the results.

In [ ]:
I_E = 1; 
I_I = 1;

w_EI = w_IE = w_II = 0;

w_EE = 0.5;

[r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);

plt.figure()
plt.plot(T, r_E, 'r-')
plt.plot(T, r_I, 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('Time')
plt.ylabel('Activity')

What happens when w_EE reaches 1 - or exceeds 1?
Can you explain why this happens?

In [ ]:
# CODE HERE



Excitatory connectivity in cortical networks can amplify the feedforward signal, and help with processing like motor movement, learning and memory, by generating persistent activity. 
In moderate values, w_EE can lead to amplification of the input, as you saw in previous examples.
But too much recurrent excitation can lead to pathological states of hyperactivity and hypersynchrony, for example as in epilepsy.

In the Wilson-Cowan model above, when w_EE > 1, the dynamics of excitatory population alone becomes unstable.
This is due to an unstable "positive feedback".
To balance this strong positive feedback, we need strong negative feedback.
This can be achieved by turning on recurrent excitatory-inhibitory connections.
Increase w_IE, w_EI and w_II to the same level as w_EE and see if this can bring the network back to balance.

In [ ]:
# CODE HERE


## Effect of time constants of integration on neuronal dynamics

As we saw in the previous exercise, strong negative feedback can balance strong positive feedback.

But for this to work properly, the inhibitory feedback should be fast.

So far, we chose similar time constants of integration for excitatory and inhibitory populations: tau_E = tau_I.

Let's now explore the effect of changing time constants on neural dynamics.

Keep tau_E = 10 as before and increase tau_I to 20, 30, 40, ...

What do you observe?

[Increase the time of simulation (T_end = 1000) to be able to make longer observations.]

In [ ]:
# CODE HERE

T_end = 1000;

tau_E = 10;

tau_I = 40;

[r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);

plt.figure()
plt.plot(T, r_E, 'r-')
plt.plot(T, r_I, 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('Time')
plt.ylabel('Activity')

For the combination (tau_E, tau_I) = (10, 40), repeat the experiment with different recurrent couplings.

Let w_EE = w_IE = J, and w_IE = w_II = -J. This means that the strength of all recurrent connections is the same (up to different signs for excitatory and inhibitory connections). We have therefore a single parameter, J, which controls the strength of recurrent coupling.

Now change J from 0.5 to 2, and repeat the experiment.

In [ ]:
tau_E = 10;
tau_I = 40;

J = 2;

w_EE = w_IE = J;
w_EI = w_II = -J;

[r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);

plt.figure()
plt.plot(T, r_E, 'r-')
plt.plot(T, r_I, 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('Time')
plt.ylabel('Activity')

In networks with strong recurrent coupling, oscillations emerge when the inhibitory feedback is delayed.
In a way, these oscillations can be seen as a sort of instability too - a transient instability, until inhibition is strong enough to control the build up of excitation (push-pull effect).

In contrast, the networks we modelled before with comparable time constants of excitation and inhibition reached a stationary state - even if they had initial transients.
Technically, the activity of excitatory and inhibitory neurons in oscillatory networks never reach to a "steady state" - as they are always transitioning between up and down states. 

You can visualize these different dynamics by plotting excitatory and inhibitory activity against each other.

Use these visualizations for different combinations of parameters and describe your observations.
when the network activity reaches a steady state, you have a fixed point where both r_E and r_I converge to.
When the network is oscillating, this plot does not converge into a single fixed point; instead, the activity converges into a cycle where r_E and r_I values periodically change over it endlessly (limit cycle).

In [ ]:
tau_E = 10;
tau_I = 40;

J = 2.0;

w_EE = w_IE = J;
w_EI = w_II = -J;

[r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);

plt.figure()
plt.plot(r_E, r_I)
plt.xlabel('r_E')
plt.ylabel('r_I')

As quantification of brain dynamics shows oscillations emerging at different frequency bands, it is interesting to ask whether the brain is ever "stationary".
As we saw before, one way to generate these oscillations is from the interaction of excitation and inhibition in brain networks. In this case, the strength and the frequency of oscillations would be determined by the strength of recurrent and feedback connections, and the timescale of their responses and communication delays. One source of this delay can, for instance, be the synaptic delays, but we have not modelled them here (there are other potential mechanisms for oscillations, like gap junctions or short-term plasticity, which can be modelled in more advanced models).

Within the framework of Wilson-Cowan model, can you tell how we can change the parameters to control the frequency of oscillations?
How would you change the time constants to get slower or faster oscillations?

In [ ]:
tau_E = 5;
tau_I = 10;

J = 5;

w_EE = w_IE = J;
w_EI = w_II = -J;

[r_E, r_I, T] = WC_solver(I_E, I_I, w_EE, w_EI, w_IE, w_II, tau_E, tau_I, dt, T_end);

plt.figure()
plt.plot(T, r_E, 'r-')
plt.plot(T, r_I, 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('Time')
plt.ylabel('Activity')

We can quantify these oscillations in our model networks by using Fourier transform.
It calculates the strength of oscillations at each frequency.

Use the function which is provided below to find the main oscillation frequency of the previous network.

In [ ]:
def FF(x0, plotting):
    f_s = 1000/dt

    x = x0 - np.nanmean(x0)

    X = fftpack.fft(x)
    freqs = fftpack.fftfreq(len(x)) * f_s

    max_freq = freqs[np.argmax(abs(X))]
    max_mag = abs(X[np.argmax(abs(X))])

    if plotting: 
        plt.figure()
        plt.stem(freqs, np.abs(X))
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Frequency Domain (Spectrum) Magnitude')
        plt.xlim([0,40])
    return freqs, X, max_freq, max_mag

[freqs, X, max_freq, max_mag] = FF(r_E, 1)

print(max_freq)

Change the parameters of network (time constants and recurrent coupling) and see how the main frequency changes.

In [ ]:
## CODE HERE


## Simulating large-scale excitatory-inhibitory networks

So far, we gained some computational insights into the interaction of excitation and inhibition in a reduced model composed of two populations.
Although useful for understanding and analysis, reduced models are unlikely to tell us everything about the complex dynamics of large-scale networks in the brain.
We therefore need to expand our models to simulate multiple networks of neurons.

The previous implementation is, however, difficult to scale with the number of populations.
(As an exercise, try to expand the function *WC_solver* to 10 populations.
You need to write a function with 122 inputs (10 inputs, 10 time constants, 100 weights, in addition to dt and T_end), which returns 11 outputs. You admit it is not the most fun thing to do.)
This is not only inconvenient in terms of programming, but it is also costly in terms of computational load.
(For 10 populations, you need to solve 10 separate equations in each step of the for loop, meaning that the total run time would be slower by ~10 times -- imagine what happens when you simulate a network with N = 1000.)

Luckily, a better and smarter way is available to solve the N-dimensional differential equations.
We can use matrix-wise and vectorized operations to implement and solve these equations more easily and more efficiently.

Below is one such implementation. Can you work out how it works, and relate it to different components we had before?

In [ ]:
# - solver function (for Wilson-Cowan equations - matrix-wise implementation)
def WC_solver_MW(I, W, tau, dt, T_end):

    N = len(W);

    T = np.arange(0, T_end, dt);

    # vector of all rates
    r = np.zeros((N,len(T)))

    # vector of all inputs
    #I = 1+pl.rand(2,len(T))

    for i in range(len(T)-1):  
        I_rec = np.array(np.matrix(W) * np.matrix(r[:,i]).T).T[0]
        dr = dt/tau * (-r[:,i] + act_fun(I_rec + I))

        r[:,i+1] = r[:,i] + dr

    return r, T

In the following code, we used the matrix-wise implementation to simulate the same networks as before, with 2 populations.
Note that the weight matrix should be provided as a 2 x 2 array (N x N in general; this will be converted to a matrix within the function to perform matrix-wise multiplication), and other parameters like the time constants and inputs are provided as 1 x N arrays (which is a format for vectorized operation in Python).

Using vectorized representations and matrix-wise operations, all the N-dimensional equations can be solve in one line (at each step of the for loop); observe and identify this in the function above.

In [ ]:
W = np.array([[w_EE, w_EI],
                [w_IE, w_II]]);

tau = np.array([tau_E, tau_I])

I = [1,1];

[r, T] = WC_solver_MW(I, W, tau, dt, T_end);

plt.figure()
plt.plot(T, r[0,:], 'r-')
plt.plot(T, r[1,:], 'b-')
plt.legend(['Exc', 'Inh'])
plt.xlabel('Time')
plt.ylabel('Activity')

Experiment with the code above and repeat some of experiments you performed before with the previous implementation.
Do you get the same results?

In [ ]:
## CODE HERE


The real efficiency of the new implementation is demonstrated for larger number of populations / equations.
In the following, we use the function to simulate the dynamics of activity of N interacting populations.
Instead of specifying the connection weights between all pairs of pre- and post-synaptic populations, we draw the weights from a distribution - in this case a Gaussian distribution with a certain mean and standard deviation.

In [ ]:
N = 10;

W = np.random.normal(0, 1, [N,N]) / np.sqrt(N)

tau = 10*np.ones(N);

I = np.ones(N);

[r, T] = WC_solver_MW(I, W, tau, dt, T_end);

plt.figure(figsize=(15,5))

plt.subplot(121)
plt.imshow(W)
plt.colorbar()
plt.title('Connection weights')
plt.xlabel('Pop #')
plt.ylabel('Pop #')

plt.subplot(122)
plt.plot(T, r.T, '-')
plt.xlabel('Time')
plt.ylabel('Activity')

Use the code above to simulate the dynamics of large-scale networks for different weight distributions.
You can start by changing the mean and std of the Gaussian distribution and describe what you observe.
You can also try other distributions, like log normal distribution, which seems to be better for describing the long-tailed distribution of weights in the brain.

In [ ]:
## CODE HERE


# Day 8 Challenge: analysis of neural dynamics in networks with imbalance of excitation and inhibition



Many brain diseases are described by the imbalance of excitation and inhibition. Modelling and analysing the neural dynamics emerging in these dysfunctional networks could be key to understanding and maybe curing the pathological states.

In the tutorial we learned about how to simulate and analyse the dynamics of networks with excitatory-inhibitory balance.
The aim of this challenge is to investigate how imbalance of excitation and inhibition in brain networks can affect neuronal activity, what the signatures of this imbalance are in neuronal dynamics, and how potential interventions can alleviate them.

Use the codes developed in the tutorial, and perform the experiments outlined in the following exercises.

**Exercise 1: Excitatory-inhibitory (im)balance and hyperactivity** 

Fix other parameters of Wilson-Cowan equations (I_E = I_I = 1; tau_E = 10; tau_I = 10; dt = 0.1; T_end = 1000), and systematically change the parameters of connectivity between excitatory and inhibitory populations (w).
Instead of a four-dimensional space (w_EE, w_IE, w_IE, w_II), explore this in a reduced, two-dimensional space, characterized by the following parameters: 
1) J_E (strength of excitatory connectivity): w_EE = w_IE = J_E
2) J_I (strength of inhibitory connectivity): w_IE = w_II = -J_I

Change these two parameters from 0.1 to 1.5 systematically (J_E = [0.1, ..., 1.5], J_I = [0.1, ..., 1.5]). 
For each combination, measure the steady state activity of the excitatory and inhibitory units at the end of simulation.
Plot the activity of each population for different combination of J_E and J_I parameters (Hint: you can use heat maps, using *imshow*).

Describe and analyse when the network becomes unstable in its activity (e.g. as defined by the hyperactivity of the excitatory population).
[Hint: Very large values of activity, especially when the network becomes unstable, will mask the visibility of lower rates. You may therefore want to clip the activity to certain upper limit (try different values, e.g. 2 or 10) for visualization.]

**Exercise 2: Excitatory-inhibitory (im)balance and oscillations** 

Fix other parameters of Wilson-Cowan equations (I_E = I_I = 1; J_E = J_I = 2; dt = 0.1; T_end = 1000), and systematically change the time constants of excitation and inhibition. 
Start by choosing tau_E = 5, and study the effect of changing tau_I from 5 to 40 (tau_I = [5, ..., 40]) on network oscillations.
For each combination, visualize the activity of the network and its oscillations (or lack thereof).
How does the frequency of oscillations change by increasing tau_I? How does the magnitude/ strength of oscillations change? 

[*Bonus*: Measure the oscillations in each network by quantifying the main frequency and its magnitude (using the function from tutorial).
Plot them both and describe what the two graphs demonstrate.
(Hint: Plot the magnitude of the spectrum in logarithmic scale, to be able to compare changes across time constants.)]

How do the results depend on recurrent coupling (J) and time constant of excitation (tau_E)?
- Change J to 1 and 3 (J_E=J_I=1 and J_E=J_I=3), while the rest of parameters are the same as before, and repeat the experiments. 
- Change tau_E to 2 and 10, while the rest of parameters are the same as before, and repeat the experiments. 
Describe and explain your observations.

**Exercise 3: Excitatory-inhibitory (im)balance and interventions** 

Investigate how we can manipulate the network to reduce pathological oscillations.
Start with a Wilson-Cowan network from previous exercise with strong oscillations.
Choose: I_E = I_I = 1; tau_E = 5, tau_I = 15; J_E = J_I = 2; dt = 0.1; T_end = 1000.

As we saw in the previous exercise and in the tutorial, the obvious fix to reduce oscillations in this model was to make inhibition faster (smaller tau_I).
Apart from that, explore the effect of other manipulations on oscillations:
- Increase the external input to the inhibitory population (try: I_I = [1, ..., 2]).
- Increase the strength of I-to-E connection weights (try: J_I (=J_IE=J_II) = [2, ..., 3]).
- [Bonus: Increase the excitability of inhibitory neurons (you need to tweak the activation function to achieve that).]

Describe how each manipulation changes the oscillations.
What are the pros and cons of each manipulation to you?

**Writing:** 
Based on Ex. 1-3, write up your understanding of the following questions:
1) In which ways can the dynamics of E/I balanced networks in the brain become unstable? (compare average activity with temporal dynamics)
2) In which ways may inhibition fail, and therefore lead to E/I imbalance? (compare the strength of inhibitory feedback with its temporal delay)
3) In which ways will we be potentially able to restore the E/I balance, by manipulating the inhibitory populations?